In [ ]:
import os
import time
from tqdm import tqdm
import itertools
import openai
openai.api_key = ''
import pandas as pd

In [ ]:
def process_jobs(input_folder, output_folder, description, prompt):
    
    jobs = os.listdir(input_folder)
    jobs = sorted(jobs, key=lambda x: int(x.split('_')[-1].split('.')[0]))

    for job in tqdm(jobs, desc=description):

        # Load the job posting
        file_path = os.path.join(input_folder, job)
        with open(file_path, encoding='utf-8') as f:
            content = f.read()

        # Init the rate limit timer
        start_time = time.time()
        tokens_per_minute = 90000
        tokens_used = 0

        # Format the prompt
        modified_prompt = prompt.format(content=content)

        # Call the ChatCompletionsAPI
        output_path = os.path.join(output_folder, job)
        with open(output_path, 'w', encoding='utf-8') as f:
            completion = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": modified_prompt}])

            try:
                f.write(completion.choices[0].message['content'])

                # Check rate limits and wait if necessary
                tokens_used += completion['usage']['total_tokens']
                elapsed_time = time.time() - start_time

                if elapsed_time >= 61:
                    start_time = time.time()
                    tokens_used = 0

                if tokens_used >= tokens_per_minute:
                    print('\nWaiting to avoid rate limit')
                    time.sleep(61)

            except IndexError:
                print(f'API Error')
                break

## Skills

In [ ]:
prompt = """
Bitte anwtorte mit einer Liste von Skills wie Programmiersprachen, Softwareprodukte und Softwarebibliotheken die in dieser Stellenauschreibung für eine Data-related Stelle erwähnt werden.
Sei sorgfältig und lese die Stellenauschreibung Schritt-für-Schritt.
Wenn in der Stellenauschreibung nichts davon erwähnt wird, dann antworte mit NONE. Antworte nur mit einer komma-getrennten Liste von Skills.

Beispiele für Skills:
Machine Learning, Python, SQL, Azure, PowerBI

Skill Liste Ausgabeformat:
Skill1, Skill2, Skill3, Skillx

Stellenauschreibung:
###
{content}
###
"""

In [ ]:
input_folder = 'Data/Scraping/Jobs'
output_folder = 'Data/Wrangling/Skills'

In [ ]:
process_jobs(input_folder, output_folder, 'Extracting skills', prompt)

## Salaries

In [ ]:
prompt = """
Please extract the salary and the payment period from the following job posting. Format it according to the examples listed below. If there is a range write the lower boundary of the range. Write it with a precision if two decimal places. Write no points or semicolons for thousands. Read through the text step by step. If there is no salary mentioned just write NONE.

Examples Unformatted:
1. The salary consists of EUR 75,000.00/year
2. Offer a market-compliant salary of EUR 4,000.23 gross per month.
3. We pay 3500-4000/monthly
4. Salary of 50000

Examples Formatted:
1. 75000.00;Yearly
2. 4000.23;Monthly
3. 3500.00;Monthly
4. 50000.00;Yearly

Job Posting:
###
{content}
###
"""

In [ ]:
input_folder = 'Data/Scraping/Jobs'
output_folder = 'Data/Wrangling/Salaries'

In [ ]:
process_jobs(input_folder, output_folder, 'Extracting salaries', prompt)

## DataFrame

### Salaries

In [ ]:
input_folder = 'Data/Wrangling/Salaries'

In [ ]:
jobs = os.listdir(input_folder)
jobs = sorted(jobs, key=lambda x: int(x.split('_')[-1].split('.')[0]))

salaries = []
for i, job in enumerate(jobs):
    
    file_path = os.path.join(input_folder, job)
    with open(file_path) as f:
        content = f.read()
        
    if 'NONE' in content or 'None' in content:
        salaries.append('NONE')
        continue

    try:
        # Retrieve salary
        salary = content.split(';')[0]
        if '-' in salary:
            salary = salary.split('-')[0]

        salary = salary.replace(',', '')
        salary = salary.strip()

        # Retrieve period
        period = content.split(';')[1]
        if period == 'Monthly':
            salary = round(float(salary)*14,2)

        elif period == 'Yearly':
            salary = round(float(salary), 2)
            
        salaries.append(salary)  

    except:
        print(f'Error processing job {i+1}') 
        
len(salaries)

### Skills

In [ ]:
input_folder = 'Data/Wrangling/Skills'

In [ ]:
jobs = os.listdir(input_folder)
jobs = sorted(jobs, key=lambda x: int(x.split('_')[-1].split('.')[0]))

skills = []
for i, job in enumerate(jobs):
    
    # Load the extracted skills
    file_path = os.path.join(input_folder, job)
    with open(file_path, encoding='utf-8') as f:
        content = f.read()
        
    skills.append(content)

# Skills cleaning
skills = [skills.split(',') for skills in skills]
skills = [[word for skill in job for word in skill.split('/')] for job in skills]
skills = [[skill.lower() for skill in job] for job in skills]
skills = [[skill.replace(' ', '').replace('-', '') for skill in job] for job in skills]
skills = [[skill.replace('ms', '').replace('microsoft', '') for skill in job] for job in skills]
skills = [[skill.replace('google', '') for skill in job] for job in skills]
skills = [[skill.replace('(', '').replace(')', '') for skill in job] for job in skills]
skills = [[skill.strip() for skill in job] for job in skills]
skills = [[skill for skill in job if skill] for job in skills]

In [ ]:
# DF columns
columns = list(itertools.chain(*skills))
columns = set(columns)
columns = list(columns)
columns.append('Salary')

df = pd.DataFrame(columns=columns)

# DF rows
rows = []
for i, job in enumerate(skills):

    # We init a new empty row
    row = pd.Series([False]*(len(columns)), index=df.columns)

    # We fill the row with the one-hot encoded skills 
    for skill in job:
        if skill in columns:
            row[skill] = True

    if salaries[i] == 'NONE':
        row['Salary'] = False
    else:  
        row['Salary'] = salaries[i]

    rows.append(row)
    
df = pd.DataFrame(rows)
df = df.drop(columns=['none'])
df.to_csv('Data/Wrangling/jobs_df.csv', index=False)